# Device

In [ ]:
import torch                         #Включаем видеокарту если есть
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используется устройство: {device}")

# Grid search hyperparam

In [ ]:
import pandas as pd                                         #+ 2 доп.гиперпараметра
from neuralprophet import NeuralProphet
from itertools import product
from sklearn.metrics import mean_absolute_error
from joblib import Parallel, delayed  # Для параллельных вычислений

# Блок 1: Загрузка данных и предварительная обработка
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data[(data['Дата'] >= '2022-01-01')]

# Подготовка данных для модели NeuralProphet
data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].str.replace(',', '.').astype(float)

# Удаление дубликатов по колонке 'ds' (даты) для каждого магазина
data = data.drop_duplicates(subset=['store', 'ds'])

# Блок 2: Определение сетки гиперпараметров
n_changepoints = [5, 10, 20, 30]
changepoints_range = [0.75, 0.85, 0.95]
yearly_seasonality = [True, 5, 10]
weekly_seasonality = [True, 3, 6]
seasonality_mode = ['additive', 'multiplicative']
trend_reg = [0.4]
seasonality_reg = [0.4]

# Минимальное количество данных для расчета гиперпараметров (например, 30 дней)
min_data_points = 60

# Функция для расчета гиперпараметров для каждого магазина
def calculate_best_params_for_store(store_id, store_data):
    if len(store_data) < min_data_points:
        print(f"Пропуск магазина {store_id}: недостаточно данных ({len(store_data)} строк)")
        return None
    
    best_accuracy = float('inf')  # Начальная высокая точность (для MAE)
    best_params = {}
    
    # Разделение данных на тренировочные и тестовые
    train_size = int(len(store_data) * 0.8)
    train_data = store_data[:train_size]
    test_data = store_data[train_size:]
    
    # Перебор всех комбинаций гиперпараметров
    for n_cp, cp_range, yearly_seas, weekly_seas, seas_mode, t_reg, s_reg in product(
        n_changepoints, changepoints_range, yearly_seasonality, weekly_seasonality, seasonality_mode, trend_reg, seasonality_reg
    ):
        # Создание и обучение модели
        model = NeuralProphet(
            n_changepoints=n_cp,
            changepoints_range=cp_range,
            yearly_seasonality=yearly_seas,
            weekly_seasonality=weekly_seas,
            seasonality_mode=seas_mode,
            trend_reg=t_reg,
            seasonality_reg=s_reg
            #device=device
        )
        
        model.fit(train_data, freq='D')
        
        # Прогнозирование на тестовых данных
        future = model.make_future_dataframe(test_data, periods=len(test_data))
        forecast = model.predict(future)
        
        # Оценка точности
        y_true = test_data['y'].values
        y_pred = forecast['yhat1'].values[:len(test_data)]
        accuracy = mean_absolute_error(y_true, y_pred)
        
        # Сравнение точности и обновление лучших параметров
        if accuracy < best_accuracy:
            best_accuracy = accuracy
            best_params = {
                'store': store_id,
                'n_changepoints': n_cp,
                'changepoints_range': cp_range,
                'yearly_seasonality': yearly_seas,
                'weekly_seasonality': weekly_seas,
                'seasonality_mode': seas_mode,
                'trend_reg': t_reg,
                'seasonality_reg': s_reg
            }
    
    return best_params

# Получаем список всех магазинов
stores = data['store'].unique()

# Параллельный расчет для всех магазинов
best_params_list = Parallel(n_jobs=-1)(
    delayed(calculate_best_params_for_store)(store_id, data[data['store'] == store_id][['ds', 'y']]) 
    for store_id in stores
)

# Убираем None значения (если магазин был пропущен из-за недостатка данных)
best_params_list = [params for params in best_params_list if params is not None]

# Блок 3: Сохранение гиперпараметров для всех магазинов в Excel файл
best_params_df = pd.DataFrame(best_params_list)
best_params_df.to_excel('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/best_model_params_all_stores+.xlsx', index=False)

# Bayes optimization hyperparam

In [ ]:
import pandas as pd
import optuna
from neuralprophet import NeuralProphet
from sklearn.metrics import mean_absolute_error
from joblib import Parallel, delayed

# 🔹 Загружаем данные
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data[(data['Дата'] >= '2023-01-01')]

# 🔹 Переименование колонок
data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].str.replace(',', '.').astype(float)
data = data.drop_duplicates(subset=['store', 'ds'])

# 🔹 Минимальное количество данных для расчета гиперпараметров
min_data_points = 60

# 🔥 Функция для оптимизации гиперпараметров
def objective(trial, store_data, store_id):
    if len(store_data) < min_data_points:
        return float('inf')  # Пропускаем магазины с недостатком данных

    # Выбор гиперпараметров
    n_changepoints = trial.suggest_int('n_changepoints', 5, 30)  
    changepoints_range = trial.suggest_float('changepoints_range', 0.75, 0.95)
    yearly_seasonality = trial.suggest_int('yearly_seasonality', 1, 20)  
    weekly_seasonality = trial.suggest_int('weekly_seasonality', 1, 7)  
    seasonality_mode = trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative'])
    trend_reg = trial.suggest_float('trend_reg', 0.1, 1.0)
    seasonality_reg = trial.suggest_float('seasonality_reg', 0.1, 1.0)
    growth = trial.suggest_categorical('growth', ['linear', 'discontinuous'])
    n_lags = trial.suggest_int('n_lags', 0, 30)

    # 🔹 Устанавливаем ar_reg ТОЛЬКО если n_lags > 0
    if n_lags > 0:
        ar_reg = trial.suggest_float('ar_reg', 0.0, 1.0)
    else:
        ar_reg = None  # NeuralProphet автоматически игнорирует None

    # 🔹 Разделение данных на train/test
    train_size = int(len(store_data) * 0.8)
    train_data, test_data = store_data[:train_size], store_data[train_size:]

    # 🔹 Обучение модели
    model = NeuralProphet(
        n_changepoints=n_changepoints,
        changepoints_range=changepoints_range,
        yearly_seasonality=yearly_seasonality,
        weekly_seasonality=weekly_seasonality,
        seasonality_mode=seasonality_mode,
        trend_reg=trend_reg,
        seasonality_reg=seasonality_reg,
        growth=growth, 
        n_lags=n_lags, 
        ar_reg=ar_reg if ar_reg is not None else 0.0  # Устанавливаем 0.0, если ar_reg не задан
    )
    model.fit(train_data, freq='D')

    # 🔹 Генерация будущих дат
    future = model.make_future_dataframe(df=train_data, periods=len(test_data))

    # 🔹 Получение прогноза
    forecast = model.predict(future)

    # 🔹 Проверка длины прогноза
    if len(forecast) < len(test_data):
        return float('inf')  # Если прогноз короче тестовой выборки, игнорируем итерацию

    # 🔹 Оценка модели
    y_true, y_pred = test_data['y'].values, forecast['yhat1'].values[:len(test_data)]
    mae = mean_absolute_error(y_true, y_pred)

    return mae

# 🔥 Функция для оптимизации гиперпараметров по каждому магазину
def optimize_for_store(store_id, store_data):
    print(f"🔄 Оптимизация для магазина {store_id}...")
    
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, store_data, store_id), n_trials=50)  # Количество итераций

    best_params = study.best_params
    best_params['store'] = store_id

    print(f"✅ Лучшие параметры для магазина {store_id}: {best_params}")
    
    return best_params

# 🔹 Получение списка магазинов
stores = data['store'].unique()

# 🔥 Параллельная оптимизация для ускорения
best_params_list = Parallel(n_jobs=-1)(
    delayed(optimize_for_store)(store_id, data[data['store'] == store_id][['ds', 'y']]) for store_id in stores
)

# 🔹 Сохранение лучших параметров
best_params_df = pd.DataFrame(best_params_list)
best_params_df.to_excel('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/best_model_params_bo.xlsx', index=False)

print("✅ Оптимизация завершена! Результаты сохранены в best_model_params_bo.xlsx.")


# Grid

In [ ]:
# 52488 комбинаций перебора.для 1 магазина при условии паралельных вычислений 16 комбинаций в минуту 
# на процесооре будет считать 54 часа...
param_grid = {
    'n_changepoints': [5, 10, 20],  # Количество точек изменения тренда
    'changepoints_range': [0.8, 0.9, 1.0],  # Диапазон для выбора точек изменения
    'yearly_seasonality': [True, 5, 10],  # True - авто, 5 или 10 для настройки
    'weekly_seasonality': [True, 3, 5],  # True - авто, или точные значения
    'daily_seasonality': [False, 10, 20],  # Добавляем дневную сезонность, если она есть
    'seasonality_mode': ['additive', 'multiplicative'],  # Режим сезонности
    'growth': ['linear', 'discontinuous'],  # Режим роста
    'n_lags': [0, 5, 10],  # Задержки для добавления автокорреляции
    'ar_regularization': [0, 0.1, 0.5],  # Регуляризация для автокорреляции
    'trend_reg': [0, 0.1, 0.5],  # Регуляризация для тренда
    'seasonality_reg': [0, 0.1, 0.5],  # Регуляризация для сезонности
}

# Рассчет прогноза с гиперпараметрами Grid Search

In [ ]:
import warnings                              # +2 новых гипера
import pandas as pd
from neuralprophet import NeuralProphet
from joblib import Parallel, delayed

warnings.filterwarnings("ignore")

# Блок 1: Загрузка данных и предварительная обработка
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data[(data['Дата'] >= '2023-01-01')]

# Подготовка данных для модели NeuralProphet
data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].str.replace(',', '.').astype(float)
data.dropna(inplace=True)

# Блок 2: Загрузка гиперпараметров модели
try:
    model_params = pd.read_excel('best_model_params_bayesian.xlsx')
except FileNotFoundError:
    model_params = pd.DataFrame(columns=['store', 'n_changepoints', 'changepoints_range', 'yearly_seasonality', 
                                         'weekly_seasonality', 'seasonality_mode', 'trend_reg', 'seasonality_reg'])

# Блок 3: Предпраздничные даты и исключенные магазины
pre_holiday_dates = ['2025-02-22','2025-03-07','2025-05-08','2025-06-11','2025-04-20','2025-05-02','2025-05-03',
                     '2024-12-28','2024-12-29','2024-12-30','2024-12-31','2025-12-28','2025-12-29','2025-12-30','2025-12-31']
excluded_stores = [149, 155, 164, 111, 123, 136, 127, 151, 109]
standard_stores = [187, 188]

# Функция для прогнозирования для одного магазина
def forecast_store(store):
    if store in excluded_stores:
        return pd.DataFrame()

    store_data = data[data['store'] == store]
    if len(store_data) < 2:
        print(f'Пропускаем магазин {store} из-за недостатка данных')
        return pd.DataFrame()

    store_data = store_data.drop_duplicates(subset=['ds'])
    store_data = store_data.drop(columns=['store'])

    # Проверка, используется ли магазин в стандартных гиперпараметрах
    if store in standard_stores:
        model = NeuralProphet()
    else:
        params = model_params[model_params['store'] == store]
        if params.empty:
            print(f'Не найдены гиперпараметры для магазина {store}, пропускаем его')
            return pd.DataFrame()

        model = NeuralProphet(
            n_changepoints=params['n_changepoints'].values[0],
            changepoints_range=params['changepoints_range'].values[0],
            yearly_seasonality=params['yearly_seasonality'].values[0],
            weekly_seasonality=params['weekly_seasonality'].values[0],
            seasonality_mode=params['seasonality_mode'].values[0],
            trend_reg=params['trend_reg'].values[0],
            seasonality_reg=params['seasonality_reg'].values[0]
        )

    model.fit(store_data, freq='D')
    future = model.make_future_dataframe(store_data, periods=50)
    forecast = model.predict(future)
    forecast['store'] = store

    for pre_holiday_date in pre_holiday_dates:
        mask = forecast['ds'] == pd.to_datetime(pre_holiday_date)
        forecast.loc[mask, 'yhat1'] *= 1.165

    return forecast

# Получение уникальных ID магазинов
store_ids = data['store'].unique()

# Параллельное прогнозирование для всех магазинов
all_forecasts = Parallel(n_jobs=-1)(delayed(forecast_store)(store) for store in store_ids)

# Объединение всех прогнозов в один DataFrame
all_forecasts = pd.concat(all_forecasts, ignore_index=True)

# Сохранение результатов прогноза в новом Excel-файле
selected_columns = ['ds', 'store', 'yhat1']
all_forecasts[selected_columns].to_excel('ФевральBayas.xlsx', index=False)


# Рассчет прогноза с гиперпараметрами Bayes

In [ ]:
import warnings
import pandas as pd
from neuralprophet import NeuralProphet
from joblib import Parallel, delayed

warnings.filterwarnings("ignore")

# Блок 1: Загрузка данных и предварительная обработка
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data[data['Дата'] >= '2023-01-01']

data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].str.replace(',', '.').astype(float)
data.dropna(inplace=True)

# Блок 2: Загрузка гиперпараметров модели
try:
    model_params = pd.read_excel('best_model_params_bo.xlsx')
except FileNotFoundError:
    model_params = pd.DataFrame(columns=['store', 'n_changepoints', 'changepoints_range', 'yearly_seasonality', 
                                         'weekly_seasonality', 'seasonality_mode', 'trend_reg', 'seasonality_reg',
                                         'growth', 'n_lags', 'ar_reg'])

# Блок 3: Предпраздничные даты и исключенные магазины
pre_holiday_dates = ['2025-02-22', '2025-03-07', '2025-05-08', '2025-06-11', '2025-04-20', '2025-05-02', '2025-05-03',
                     '2024-12-28', '2024-12-29', '2024-12-30', '2024-12-31', '2025-12-28', '2025-12-29', '2025-12-30', '2025-12-31']
excluded_stores = [149, 155, 164, 111, 123, 136, 127, 151, 109]
standard_stores = [187, 188]

# Функция для прогнозирования одного магазина
def forecast_store(store):
    if store in excluded_stores:
        return pd.DataFrame()

    store_data = data[data['store'] == store]
    if len(store_data) < 2:
        print(f'Пропускаем магазин {store} из-за недостатка данных')
        return pd.DataFrame()

    if store in standard_stores:
        model = NeuralProphet()
    else:
        params = model_params[model_params['store'] == store]
        if params.empty:
            print(f'Не найдены гиперпараметры для магазина {store}, пропускаем его')
            return pd.DataFrame()

        params = params.to_dict('records')[0]  # Преобразуем в словарь

        # Учитываем возможность числового значения для seasonality
        yearly_seasonality = params['yearly_seasonality']
        if yearly_seasonality not in [True, False]:
            yearly_seasonality = int(yearly_seasonality)

        weekly_seasonality = params['weekly_seasonality']
        if weekly_seasonality not in [True, False]:
            weekly_seasonality = int(weekly_seasonality)

        model = NeuralProphet(
            n_changepoints=int(params['n_changepoints']),
            changepoints_range=float(params['changepoints_range']),
            yearly_seasonality=yearly_seasonality,
            weekly_seasonality=weekly_seasonality,
            seasonality_mode=params['seasonality_mode'],
            trend_reg=float(params['trend_reg']),
            seasonality_reg=float(params['seasonality_reg']),
            growth=params['growth'],
            n_lags=int(params['n_lags']),
            ar_reg=float(params['ar_reg'])  # Исправлено имя параметра
        )

    model.fit(store_data, freq='D')

    # Создание будущих дат
    future = model.make_future_dataframe(df=store_data, periods=50)

    # Получение прогноза
    forecast = model.predict(future)
    forecast['store'] = store

    # Корректировка предпраздничных дней
    forecast['ds'] = pd.to_datetime(forecast['ds'])
    forecast.loc[forecast['ds'].isin(pd.to_datetime(pre_holiday_dates)), 'yhat1'] *= 1.165

    return forecast

# Запуск прогнозирования
store_ids = data['store'].unique()
all_forecasts = Parallel(n_jobs=-1)(delayed(forecast_store)(store) for store in store_ids)

# Объединение прогнозов
all_forecasts = [f for f in all_forecasts if not f.empty]  # Убираем пустые DataFrame
if len(all_forecasts) > 0:
    final_forecast = pd.concat(all_forecasts, ignore_index=True)
    final_forecast[['ds', 'store', 'yhat1']].to_excel('Февраль.xlsx', index=False)
    print("Прогноз успешно сохранен в 'Февраль.xlsx'")
else:
    print("Не удалось сформировать прогноз, все магазины исключены.")


# Графики

In [ ]:
import matplotlib.pyplot as plt                     # График по всем магазинам
import matplotlib.dates as mdates
import pandas as pd

# Загрузка данных прогнозов и фактических продаж
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)

# Переименование столбцов для удобства
data = data.rename(columns={
    'Дата': 'ds', 
    'Номер Магазина': 'store', 
    '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'
})

# Преобразование столбца 'y' в числовой формат
data['y'] = data['y'].str.replace(',', '.').astype(float)

# Фильтрация данных начиная с 2024 года
data = data[data['ds'] >= '2023-12-01']

# Суммирование фактических данных по всем магазинам
total_sales = data.groupby('ds').agg({'y': 'sum'}).reset_index()

# Загрузка прогнозов
all_forecasts = pd.read_excel('Forecast_NeuralProphet_with_reg.xlsx',sheet_name='Sheet1', index_col=None)  # Измени имя файла, если нужно
all_forecasts['ds'] = pd.to_datetime(all_forecasts['ds'])
filtered_forecasts = all_forecasts[all_forecasts['ds'] >= '2023-12-01']

# Суммирование прогнозов по всем магазинам
summed_forecasts = filtered_forecasts.groupby('ds').agg({'yhat1': 'sum'}).reset_index()  # Измени 'yhat' на 'yhat1'

# Создание общего графика суммарного прогноза с фактическими данными
plt.figure(figsize=(14, 8))

# Линия фактических продаж
plt.plot(total_sales['ds'], total_sales['y'], label='Продажи', color='red', linewidth=2)

# Линия суммарного прогноза
plt.plot(summed_forecasts['ds'], summed_forecasts['yhat1'], label='Прогноз', color='blue', linestyle='--', linewidth=2)  # Измени 'yhat' на 'yhat1'

# Настройка заголовка и меток осей
plt.title('Динамика Продаж и Прогноза', fontsize=16)
plt.xlabel('Понедельники', fontsize=14)
plt.ylabel('Продажи в руб.', fontsize=14)

# Отображение легенды
plt.legend(fontsize=12)

# Отображение сетки
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Улучшение форматирования осей
plt.xticks(rotation=45)
plt.tight_layout()

# Добавление разделителей тысяч для оси Y
plt.gca().get_yaxis().set_major_formatter(plt.matplotlib.ticker.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

# Добавление меток начала каждой недели на ось X
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MO))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

# Показ графика
plt.show()


In [ ]:
import matplotlib.pyplot as plt                 # График по выбранному магазину
import matplotlib.dates as mdates
import pandas as pd

# Загрузка данных прогнозов и фактических продаж
data = pd.read_csv('C:/Users/bondarenKovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)

# Переименование столбцов для удобства
data = data.rename(columns={
    'Дата': 'ds', 
    'Номер Магазина': 'store', 
    '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'
})

# Преобразование столбца 'y' в числовой формат
data['y'] = data['y'].str.replace(',', '.').astype(float)

# Фильтрация данных начиная с 2024 года
data = data[data['ds'] >= '2023-12-01']

# Фильтрация данных по выбранному магазину (например, магазин с ID 148)
store_id = 152
store_data = data[data['store'] == store_id]

# Загрузка прогнозов
all_forecasts = pd.read_excel('Forecast_NeuralProphet_with_reg.xlsx', index_col=None)

# Вывод доступных столбцов для диагностики
print(all_forecasts.columns)  # Добавлено для отладки

# Проверка названия столбца 'ds'
if 'ds' in all_forecasts.columns:
    all_forecasts['ds'] = pd.to_datetime(all_forecasts['ds'])
else:
    all_forecasts[' Дата'] = pd.to_datetime(all_forecasts[' Дата'])  # Измени имя, если нужно

filtered_forecasts = all_forecasts[all_forecasts['ds'] >= '2023-12-01']  # Измените имя столбца при необходимости

# Фильтрация прогнозов по выбранному магазину
store_forecasts = filtered_forecasts[filtered_forecasts['store'] == store_id]

# Создание графика прогноза с фактическими данными для выбранного магазина
plt.figure(figsize=(14, 8))

# Линия фактических продаж
plt.plot(store_data['ds'], store_data['y'], label='Продажи', color='red', linewidth=2)

# Линия прогноза
plt.plot(store_forecasts['ds'], store_forecasts['yhat1'], label='Прогноз', color='blue', linestyle='--', linewidth=2)

# Настройка заголовка и меток осей
plt.title(f'Динамика Продаж и Прогноза для магазина {store_id}', fontsize=16)
plt.xlabel('Понедельники', fontsize=14)
plt.ylabel('Продажи в руб.', fontsize=14)

# Отображение легенды
plt.legend(fontsize=12)

# Отображение сетки
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Улучшение форматирования осей
plt.xticks(rotation=45)
plt.tight_layout()

# Добавление разделителей тысяч для оси Y
plt.gca().get_yaxis().set_major_formatter(plt.matplotlib.ticker.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

# Добавление меток начала каждой недели на ось X
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MO))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

# Показ графика
plt.show()
